In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import pickle

In [4]:
pd.pandas.set_option('display.max_rows', 50)
pd.pandas.set_option('display.max_columns', 1000)

In [5]:
rdf = pd.read_csv('Zomato Restaurants DataFrame - Clean & FeatEng II.csv', index_col=False)

In [6]:
rdf.head()

,Restaurant Name,Restaurant Area,Restaurant Rating,Restaurant Review,Latitude,Longitude,Website,Restaurant Address,Price for 2,Additional Info,Province,City,District/Kecamatan,Delivery Service,Place Type,Cuisine Regionality,Cuisines,acceptscryptocurrency,alkoholtersedia,areamerokok,bawapulangtersedia,biayabukabotol,biayamasuk,boardgames,brunch,buffet,byob,craftbeer,dessertsandbakes,didalamruangan,drivethrough,gastropub,hanyabawapulang,hanyabir,hanyapesanantar,hanyauntukreservasimeja,hanyawine,hanyawinedanbir,kaki5bersertifikatbpom,karaoke,kehidupanmalam,lantaidansa,liveentertainment,livemusic,makandengangrup,makanmewah,mejabilliard,menumakansiang,menyediakancocktail,menyediakanhalal,noadditionalinfo,parkirgratis,parkirvalettersedia,pemandangankota,pilihanbebasgluten,pilihanvegan,prasmanan,ramahuntukanak,ramahuntukdifabel,ramahvegetarian,reservasimejadirekomendasikan,rooftop,ruangpribaditersedia,ruangsholat,sarapan,shisha,sportsbar,tempatdudukdiluar,wifitersedia
0,Holy Smokes,Senopati,4.4,1437,-6.239143,106.807954,https://www.zomato.com/id/jakarta/holy-smokes-...,"Jl. Wolter Monginsidi No. 27, Senopati, Jakarta",500000,"['Di dalam ruangan', 'Hanya Wine dan Bir', 'Pa...",DKI Jakarta,Jakarta Selatan,Kebayoran Baru,No,['Casual Dining'],['Barat'],"['Grill House', 'Steak']",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1
1,Pempek Megaria,Cikini,4.3,461,-6.199900,106.843739,https://www.zomato.com/id/jakarta/pempek-megar...,"Metropole 21, Jl. Pegangsaan Timur No. 21, Cik...",70000,"['Bawa Pulang Tersedia', 'Di dalam ruangan']",DKI Jakarta,Jakarta Pusat,Menteng,No,['Quick Bites'],['Palembang'],['Pempek'],0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Tjikinii Lima,Cikini,4.0,391,-6.189802,106.837150,https://www.zomato.com/id/jakarta/tjikinii-lim...,"Jl. Cikini I No. 5, Cikini, Jakarta 10330",300000,"['Ruang Pribadi Tersedia', 'Tempat duduk di lu...",DKI Jakarta,Jakarta Pusat,Menteng,No,['Casual Dining'],"['Indonesia', 'Barat']",['Kopi'],0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,1
3,Gado-Gado Bon-Bin,Cikini,4.0,177,-6.192959,106.838377,https://www.zomato.com/id/jakarta/gado-gado-bo...,"Jl. Cikini 4 No. 5, Cikini, Jakarta",90000,"['Bawa Pulang Tersedia', 'Di dalam ruangan']",DKI Jakarta,Jakarta Pusat,Menteng,No,['Quick Bites'],"['Jawa', 'Indonesia']",['Makanan Sehat'],0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,RM Wong Tegal,Tanjung Duren,3.9,12,-6.182315,106.782992,https://www.zomato.com/id/jakarta/rm-wong-tega...,"Jl. Tanjung Duren Barat 6 No. 5, Tanjung Duren...",100000,"['Bawa Pulang Tersedia', 'Sarapan', 'Di dalam ...",DKI Jakarta,Jakarta Barat,Grogol Petamburan,No,['Quick Bites'],"['Jawa', 'Indonesia']",['Satay'],0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [7]:
# def convert_list(row, name): 
#     stripped = row[name].strip("[]")
#     splitted = list(stripped.split(", "))
#     final = [letter.strip("'") for letter in splitted]
#     return final

In [8]:
# rgdf['Restaurant Type'] = rgdf.apply(lambda row: convert_list(row, 'Restaurant Type'), axis=1)
# rgdf['Cuisine Type'] = rgdf.apply(lambda row: convert_list(row, 'Cuisine Type'), axis=1)
# rgdf['Cuisine Region Type'] = rgdf.apply(lambda row: convert_list(row, 'Cuisine Region Type'), axis=1)

# RECOMMENDER SYSTEM

In [9]:
rdf.head()

,Restaurant Name,Restaurant Area,Restaurant Rating,Restaurant Review,Latitude,Longitude,Website,Restaurant Address,Price for 2,Additional Info,Province,City,District/Kecamatan,Delivery Service,Place Type,Cuisine Regionality,Cuisines,acceptscryptocurrency,alkoholtersedia,areamerokok,bawapulangtersedia,biayabukabotol,biayamasuk,boardgames,brunch,buffet,byob,craftbeer,dessertsandbakes,didalamruangan,drivethrough,gastropub,hanyabawapulang,hanyabir,hanyapesanantar,hanyauntukreservasimeja,hanyawine,hanyawinedanbir,kaki5bersertifikatbpom,karaoke,kehidupanmalam,lantaidansa,liveentertainment,livemusic,makandengangrup,makanmewah,mejabilliard,menumakansiang,menyediakancocktail,menyediakanhalal,noadditionalinfo,parkirgratis,parkirvalettersedia,pemandangankota,pilihanbebasgluten,pilihanvegan,prasmanan,ramahuntukanak,ramahuntukdifabel,ramahvegetarian,reservasimejadirekomendasikan,rooftop,ruangpribaditersedia,ruangsholat,sarapan,shisha,sportsbar,tempatdudukdiluar,wifitersedia
0,Holy Smokes,Senopati,4.4,1437,-6.239143,106.807954,https://www.zomato.com/id/jakarta/holy-smokes-...,"Jl. Wolter Monginsidi No. 27, Senopati, Jakarta",500000,"['Di dalam ruangan', 'Hanya Wine dan Bir', 'Pa...",DKI Jakarta,Jakarta Selatan,Kebayoran Baru,No,['Casual Dining'],['Barat'],"['Grill House', 'Steak']",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1
1,Pempek Megaria,Cikini,4.3,461,-6.199900,106.843739,https://www.zomato.com/id/jakarta/pempek-megar...,"Metropole 21, Jl. Pegangsaan Timur No. 21, Cik...",70000,"['Bawa Pulang Tersedia', 'Di dalam ruangan']",DKI Jakarta,Jakarta Pusat,Menteng,No,['Quick Bites'],['Palembang'],['Pempek'],0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Tjikinii Lima,Cikini,4.0,391,-6.189802,106.837150,https://www.zomato.com/id/jakarta/tjikinii-lim...,"Jl. Cikini I No. 5, Cikini, Jakarta 10330",300000,"['Ruang Pribadi Tersedia', 'Tempat duduk di lu...",DKI Jakarta,Jakarta Pusat,Menteng,No,['Casual Dining'],"['Indonesia', 'Barat']",['Kopi'],0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,1
3,Gado-Gado Bon-Bin,Cikini,4.0,177,-6.192959,106.838377,https://www.zomato.com/id/jakarta/gado-gado-bo...,"Jl. Cikini 4 No. 5, Cikini, Jakarta",90000,"['Bawa Pulang Tersedia', 'Di dalam ruangan']",DKI Jakarta,Jakarta Pusat,Menteng,No,['Quick Bites'],"['Jawa', 'Indonesia']",['Makanan Sehat'],0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,RM Wong Tegal,Tanjung Duren,3.9,12,-6.182315,106.782992,https://www.zomato.com/id/jakarta/rm-wong-tega...,"Jl. Tanjung Duren Barat 6 No. 5, Tanjung Duren...",100000,"['Bawa Pulang Tersedia', 'Sarapan', 'Di dalam ...",DKI Jakarta,Jakarta Barat,Grogol Petamburan,No,['Quick Bites'],"['Jawa', 'Indonesia']",['Satay'],0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [10]:
def minimize_str(row, name): 
    lst = row[name]
    lst = lst.strip("[] '")
    lst = lst.replace("'",'')
    lst = lst.replace(" ",'')
    return lst

# 1.1 RECOMMENDER - CUISINES

In [11]:
#Create Dataframe Content Based by Cuisine
rdf_csne = rdf[['Restaurant Name', 'Place Type', 'Cuisines', 'Cuisine Regionality','Price for 2','Restaurant Address', 'Restaurant Rating','Restaurant Review','City', 'Province', 'Latitude', 'Longitude']]
rdf_csne['Cuisines STR'] = rdf_csne.apply(lambda row: minimize_str(row, 'Cuisines'), axis=1)

In [12]:
rdf_csne.head(10)

,Restaurant Name,Place Type,Cuisines,Cuisine Regionality,Price for 2,Restaurant Address,Restaurant Rating,Restaurant Review,City,Province,Latitude,Longitude,Cuisines STR
0,Holy Smokes,['Casual Dining'],"['Grill House', 'Steak']",['Barat'],500000,"Jl. Wolter Monginsidi No. 27, Senopati, Jakarta",4.4,1437,Jakarta Selatan,DKI Jakarta,-6.239143,106.807954,"GrillHouse,Steak"
1,Pempek Megaria,['Quick Bites'],['Pempek'],['Palembang'],70000,"Metropole 21, Jl. Pegangsaan Timur No. 21, Cik...",4.3,461,Jakarta Pusat,DKI Jakarta,-6.199900,106.843739,Pempek
2,Tjikinii Lima,['Casual Dining'],['Kopi'],"['Indonesia', 'Barat']",300000,"Jl. Cikini I No. 5, Cikini, Jakarta 10330",4.0,391,Jakarta Pusat,DKI Jakarta,-6.189802,106.837150,Kopi
3,Gado-Gado Bon-Bin,['Quick Bites'],['Makanan Sehat'],"['Jawa', 'Indonesia']",90000,"Jl. Cikini 4 No. 5, Cikini, Jakarta",4.0,177,Jakarta Pusat,DKI Jakarta,-6.192959,106.838377,MakananSehat
4,RM Wong Tegal,['Quick Bites'],['Satay'],"['Jawa', 'Indonesia']",100000,"Jl. Tanjung Duren Barat 6 No. 5, Tanjung Duren...",3.9,12,Jakarta Barat,DKI Jakarta,-6.182315,106.782992,Satay
5,Martabak Elin Bandung,['Dessert Parlor'],"['Martabak', 'Makanan Jalanan']",['Indonesia'],80000,"Jl. Pinang Mas III/A 8, Pondok Indah, Jakarta",3.9,26,Jakarta Selatan,DKI Jakarta,-6.265635,106.778986,"Martabak,MakananJalanan"
6,Bakso Sapi Rudi,['Quick Bites'],"['Bakso', 'Meat']",['Indonesia'],60000,"Jl. Rajawali Selatan I No. 54A, Gunung Sahari,...",3.9,22,Jakarta Pusat,DKI Jakarta,-6.144706,106.837500,"Bakso,Meat"
7,A Siang,['Quick Bites'],['Kopi'],['Chinese'],65000,"Ruko Taman Palem Lestari, Blok B16 No. 9, Jl. ...",3.9,7,Jakarta Barat,DKI Jakarta,-6.139209,106.725797,Kopi
8,Tropicale Juice Bar,['Toko Minuman'],"['Desserts', 'Jus', 'Minuman']",['Indonesia'],50000,"Ampera Garden, Jl. Ampera Raya No. 116, Ciland...",3.9,9,Jakarta Selatan,DKI Jakarta,-6.288935,106.817939,"Desserts,Jus,Minuman"
9,Sate Taichan Bang Tetew,['Quick Bites'],['Satay'],['Indonesia'],80000,"Jl. Danau Sunter Utara, Blok J 12 No. 26-28, S...",3.9,36,Jakarta Utara,DKI Jakarta,-6.137467,106.873776,Satay


In [13]:
cv= CountVectorizer()
cv_csn=cv.fit_transform(rdf_csne['Cuisines STR'])

In [14]:
cvcs_df= pd.DataFrame(cv_csn.todense(),columns=cv.get_feature_names(),index=rdf['Restaurant Name'])
cvcs_df.columns

Index(['ayam', 'bakmi', 'bakso', 'barbekyu', 'bebek', 'bubbletea', 'bubur',
       'burger', 'deli', 'desserts', 'dimsum', 'grillhouse', 'gudeg',
       'hanyaminuman', 'icecream', 'jus', 'kebab', 'kopi', 'kue',
       'makanancepatsaji', 'makananjalanan', 'makanansehat', 'martabak',
       'meat', 'minuman', 'pempek', 'pizza', 'ramen', 'roti', 'satay',
       'seafood', 'snacks', 'soto', 'steak', 'sushi', 'tea'],
      dtype='object')

In [15]:
cvcs_df

,ayam,bakmi,bakso,barbekyu,bebek,bubbletea,bubur,burger,deli,desserts,dimsum,grillhouse,gudeg,hanyaminuman,icecream,jus,kebab,kopi,kue,makanancepatsaji,makananjalanan,makanansehat,martabak,meat,minuman,pempek,pizza,ramen,roti,satay,seafood,snacks,soto,steak,sushi,tea
Restaurant Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Holy Smokes,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
Pempek Megaria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
Tjikinii Lima,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Gado-Gado Bon-Bin,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
RM Wong Tegal,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Nock Nock Cafe & Resto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Abun Takoyaki,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
A Wen Seafood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [16]:
cos_sin_csne=cosine_similarity(cv_csn)
pd.DataFrame(cos_sin_csne,index=rdf['Restaurant Name'],columns=rdf['Restaurant Name'])

0.0   
A Wen Seafood                                        0.0          0.0   
Talaga Kuring                                        0.0          0.0   
Tigalima Coffee & Kitchen                            0.0          0.0   

Restaurant Name            Oma Recipe  Soto Betawi Bang Japri  Kopi Ranger  \
Restaurant Name                                                              
Holy Smokes                  0.000000                     0.0     0.000000   
Pempek Megaria               0.000000                     0.0     0.000000   
Tjikinii Lima                1.000000                     0.0     0.707107   
Gado-Gado Bon-Bin            0.000000                     0.0     0.000000   
RM Wong Tegal                0.000000                     0.0     0.000000   
...                               ...                     ...          ...   
Nock Nock Cafe & Resto       1.000000                     0.0     0.707107   
Abun Takoyaki                0.000000                     0.0     0.000000   
A Wen Seafood                0.000000                     0.0     0.000000   
Talaga Kuring                1.000000                     0.0     0.707107   
Tigalima Coffee & Kitchen    0.707107                     0.0     1.000000   

Restaurant Name            Quick Chicken  Sate Tegal Kambing Muda 88  \
Restaurant Name                                                        
Holy Smokes                          0.0                     0.00000   
Pempek Megaria                       0.0                     0.00000   
Tjikinii Lima                        0.0                     0.00000   
Gado-Gado Bon-Bin                    0.0                     0.00000   
RM Wong Tegal                        0.0                     0.57735   
...                                  ...                         ...   
Nock Nock Cafe & Resto               0.0                     0.00000   
Abun Takoyaki                        0.0                     0.00000   
A Wen Seafood                        0.0                     0.00000   
Talaga Kuring                        0.0                     0.00000   
Tigalima Coffee & Kitchen            0.0                     0.00000   

Restaurant Name            Rumah Makan Pak Ubung  FIFI Coffee Bar & Bistro  \
Restaurant Name                                                              
Holy Smokes                                  0.0                  0.000000   
Pempek Megaria                               0.0                  0.000000   
Tjikinii Lima                                0.0                  0.707107   
Gado-Gado Bon-Bin                            0.0                  0.000000   
RM Wong Tegal                                0.0                  0.000000   
...                                          ...                       ...   
Nock Nock Cafe & Resto                       0.0                  0.707107   
Abun Takoyaki                                0.0                  0.000000   
A Wen Seafood                                0.0                  0.000000   
Talaga Kuring                                0.0                  0.707107   
Tigalima Coffee & Kitchen                    0.0                  1.000000   

Restaurant Name            Nasi Goreng Karee Abang Gandul  Kedai Sutera  \
Restaurant Name                                                           
Holy Smokes                                      0.000000      0.000000   
Pempek Megaria                                   0.000000      0.000000   
Tjikinii Lima                                    1.000000      1.000000   
Gado-Gado Bon-Bin                                0.000000      0.000000   
RM Wong Tegal                                    0.000000      0.000000   
...                                                   ...           ...   
Nock Nock Cafe & Resto                           1.000000      1.000000   
Abun Takoyaki                                    0.000000      0.000000   
A Wen Seafood                                    0.00000

In [17]:
def get_recomendation_csne(title):
    idx=rdf_csne[rdf_csne['Restaurant Name']==title].index[0]
    rest_similar=pd.Series(cos_sin_csne[idx])
    idx_similar=rest_similar.sort_values(ascending=False).head(6).index
    recommend=rdf_csne.loc[idx_similar]
    recommend.reset_index(drop=True,inplace=True)
    
    return recommend.loc[1:6][['Restaurant Name', 'Place Type', 'Cuisines', 'Cuisine Regionality','Price for 2','Restaurant Address', 'Restaurant Rating','Restaurant Review','City', 'Province', 'Latitude', 'Longitude']]

In [230]:
get_recomendation_csne('Holy Smokes')

,Restaurant Name,Place Type,Cuisines,Cuisine Regionality,Price for 2,Restaurant Address,Restaurant Rating,Restaurant Review,City,Province,Latitude,Longitude
1,Holy Smokes,['Casual Dining'],"['Grill House', 'Steak']",['Barat'],500000,"Jl. Wolter Monginsidi No. 27, Senopati, Jakarta",4.4,1438,Jakarta Selatan,DKI Jakarta,-6.238930,106.807761
2,Poka Ribs,['Casual Dining'],"['Grill House', 'Steak']",['Indonesia'],250000,"Summarecon Mall Serpong, Lantai Ground, Downto...",3.7,96,Tangerang Kabupaten,Banten,-6.241130,106.628486
3,Mamoo,['Quick Bites'],"['Grill House', 'Steak']",['Indonesia'],50000,"Jl. Raden Patah Blok Lebang Baru No.9, Ciledug...",3.4,56,Tangerang Kota,Banten,-6.245393,106.700506
4,Andakar,['Casual Dining'],"['Grill House', 'Steak']",['Indonesia'],250000,"Jl. Cilandak KKO No. 2, Cilandak, Jakarta",3.9,194,Jakarta Selatan,DKI Jakarta,-6.306459,106.814015
5,Tony Roma's,['Fine Dining'],"['Grill House', 'Steak']",['Indonesia'],750000,"UOB Plaza, Lantai Mezzanine, Jl. M. H. Thamrin...",4.6,486,Jakarta Pusat,DKI Jakarta,-6.198943,106.822410


# 1.2 RECOMMENDER - CUISINE REGIONALITY

In [19]:
rdf_csne['Cuisine Regionality STR'] = rdf_csne.apply(lambda row: minimize_str(row, 'Cuisine Regionality'), axis=1)

In [20]:
rdf_csne.head()

,Restaurant Name,Place Type,Cuisines,Cuisine Regionality,Price for 2,Restaurant Address,Restaurant Rating,Restaurant Review,City,Province,Latitude,Longitude,Cuisines STR,Cuisine Regionality STR
0,Holy Smokes,['Casual Dining'],"['Grill House', 'Steak']",['Barat'],500000,"Jl. Wolter Monginsidi No. 27, Senopati, Jakarta",4.4,1437,Jakarta Selatan,DKI Jakarta,-6.239143,106.807954,"GrillHouse,Steak",Barat
1,Pempek Megaria,['Quick Bites'],['Pempek'],['Palembang'],70000,"Metropole 21, Jl. Pegangsaan Timur No. 21, Cik...",4.3,461,Jakarta Pusat,DKI Jakarta,-6.199900,106.843739,Pempek,Palembang
2,Tjikinii Lima,['Casual Dining'],['Kopi'],"['Indonesia', 'Barat']",300000,"Jl. Cikini I No. 5, Cikini, Jakarta 10330",4.0,391,Jakarta Pusat,DKI Jakarta,-6.189802,106.837150,Kopi,"Indonesia,Barat"
3,Gado-Gado Bon-Bin,['Quick Bites'],['Makanan Sehat'],"['Jawa', 'Indonesia']",90000,"Jl. Cikini 4 No. 5, Cikini, Jakarta",4.0,177,Jakarta Pusat,DKI Jakarta,-6.192959,106.838377,MakananSehat,"Jawa,Indonesia"
4,RM Wong Tegal,['Quick Bites'],['Satay'],"['Jawa', 'Indonesia']",100000,"Jl. Tanjung Duren Barat 6 No. 5, Tanjung Duren...",3.9,12,Jakarta Barat,DKI Jakarta,-6.182315,106.782992,Satay,"Jawa,Indonesia"


In [21]:
cv= CountVectorizer()
cv_rgn=cv.fit_transform(rdf_csne['Cuisine Regionality STR'])

In [22]:
cvrg_df= pd.DataFrame(cv_rgn.todense(),columns=cv.get_feature_names(),index=rdf['Restaurant Name'])
cvrg_df.columns

Index(['aceh', 'amerika', 'amerikalatin', 'arab', 'asia', 'australian', 'bali',
       'bangka', 'barat', 'batak', 'belanda', 'betawi', 'chinese', 'eropa',
       'fusion', 'india', 'indiaselatan', 'indiautara', 'indonesia', 'itali',
       'jawa', 'jepang', 'jerman', 'kalimantan', 'kontinental', 'korea',
       'libanon', 'lombok', 'makassar', 'malaysia', 'manado', 'maroko',
       'medan', 'mediterania', 'meksiko', 'mongolia', 'noregion', 'padang',
       'palembang', 'peranakan', 'perancis', 'peruvian', 'singapura',
       'spanyol', 'sulawesi', 'sumatera', 'sunda', 'taiwanese', 'thai',
       'timurtengah', 'turki', 'vietnam', 'yogyakarta', 'yunani'],
      dtype='object')

In [23]:
cos_sin_rgn=cosine_similarity(cv_rgn)
pd.DataFrame(cos_sin_rgn,index=rdf['Restaurant Name'],columns=rdf['Restaurant Name'])

0.5     0.707107   
Tigalima Coffee & Kitchen                            0.5     0.707107   

Restaurant Name            Oma Recipe  Soto Betawi Bang Japri  Kopi Ranger  \
Restaurant Name                                                              
Holy Smokes                  0.000000                     0.0     0.000000   
Pempek Megaria               0.000000                     0.0     0.000000   
Tjikinii Lima                0.000000                     0.5     0.000000   
Gado-Gado Bon-Bin            0.000000                     0.5     0.000000   
RM Wong Tegal                0.000000                     0.5     0.000000   
...                               ...                     ...          ...   
Nock Nock Cafe & Resto       0.000000                     0.5     0.707107   
Abun Takoyaki                0.000000                     0.0     0.000000   
A Wen Seafood                0.707107                     0.0     0.000000   
Talaga Kuring                0.000000                     0.5     0.000000   
Tigalima Coffee & Kitchen    0.000000                     0.5     0.707107   

Restaurant Name            Quick Chicken  Sate Tegal Kambing Muda 88  \
Restaurant Name                                                        
Holy Smokes                     1.000000                         0.0   
Pempek Megaria                  0.000000                         0.0   
Tjikinii Lima                   0.707107                         0.5   
Gado-Gado Bon-Bin               0.000000                         1.0   
RM Wong Tegal                   0.000000                         1.0   
...                                  ...                         ...   
Nock Nock Cafe & Resto          0.000000                         0.5   
Abun Takoyaki                   0.000000                         0.0   
A Wen Seafood                   0.000000                         0.0   
Talaga Kuring                   0.000000                         0.5   
Tigalima Coffee & Kitchen       0.000000                         0.5   

Restaurant Name            Rumah Makan Pak Ubung  FIFI Coffee Bar & Bistro  \
Restaurant Name                                                              
Holy Smokes                             0.000000                  0.577350   
Pempek Megaria                          0.000000                  0.000000   
Tjikinii Lima                           0.000000                  0.816497   
Gado-Gado Bon-Bin                       0.408248                  0.408248   
RM Wong Tegal                           0.408248                  0.408248   
...                                          ...                       ...   
Nock Nock Cafe & Resto                  0.000000                  0.816497   
Abun Takoyaki                           0.000000                  0.000000   
A Wen Seafood                           0.408248                  0.000000   
Talaga Kuring                           0.000000                  0.408248   
Tigalima Coffee & Kitchen               0.000000                  0.816497   

Restaurant Name            Nasi Goreng Karee Abang Gandul  Kedai Sutera  \
Restaurant Name                                                           
Holy Smokes                                      0.000000      0.000000   
Pempek Megaria                                   0.000000      0.000000   
Tjikinii Lima                                    0.707107      0.707107   
Gado-Gado Bon-Bin                                0.707107      0.707107   
RM Wong Tegal                                    0.707107      0.707107   
...                                                   ...           ...   
Nock Nock Cafe & Resto                           0.707107      0.707107   
Abun Takoyaki                                    0.000000      0.000000   
A Wen Seafood                                    0.000000      0.000000   
Talaga Kuring                                    0.707107      0.707107   
Tigalima Coffee & Kitchen              

In [24]:
def get_recomendation_region(title):
    idx=rdf_csne[rdf_csne['Restaurant Name']==title].index[0]
    rest_similar=pd.Series(cos_sin_rgn[idx])
    idx_similar=rest_similar.sort_values(ascending=False).head(6).index
    recommend=rdf_csne.loc[idx_similar]
    recommend.reset_index(drop=True,inplace=True)
    
    return recommend[1:6][['Restaurant Name', 'Place Type', 'Cuisines', 'Cuisine Regionality','Price for 2','Restaurant Address', 'Restaurant Rating','Restaurant Review','City', 'Province', 'Latitude', 'Longitude']]

In [231]:
get_recomendation_region('Pempek Megaria')

,Restaurant Name,Place Type,Cuisines,Cuisine Regionality,Price for 2,Restaurant Address,Restaurant Rating,Restaurant Review,City,Province,Latitude,Longitude
1,Pempek Sriwijaya,['Quick Bites'],['Pempek'],['Palembang'],90000,"Jl. Hadiah 1, Blok D14 No. 1204 B, Jelambar, J...",3.5,18,Jakarta Barat,DKI Jakarta,-6.161505,106.783709
2,Pempek Ekspress,['Food Court'],['Pempek'],['Palembang'],70000,"Lippo Mall Puri @ The St. Moritz, Lantai Lower...",2.8,6,Jakarta Barat,DKI Jakarta,-6.188875,106.738983
3,Pempek Alik,['Quick Bites'],['Pempek'],['Palembang'],60000,"Jl. Kedoya Duri No. 24B, Kedoya, Jakarta",3.7,5,Jakarta Barat,DKI Jakarta,-6.181134,106.762256
4,Pempek Cawan Putih,['Quick Bites'],['Pempek'],['Palembang'],80000,"Jl. Pecenongan No. 16, Pecenongan, Jakarta",3.0,10,Jakarta Pusat,DKI Jakarta,-6.166608,106.826504
5,Pempek Emak Seroja,['Quick Bites'],['Pempek'],['Palembang'],100000,"Jl. Ratu Kemuning Blok D2 No. 26C, Green Ville...",3.2,6,Jakarta Barat,DKI Jakarta,-6.169589,106.769300


# 1.3 RECOMMENDER - PLACE TYPE

In [26]:
rdf_csne['Place Type STR'] = rdf_csne.apply(lambda row: minimize_str(row, 'Place Type'), axis=1)

In [27]:
cv= CountVectorizer()
cv_plc=cv.fit_transform(rdf_csne['Place Type STR'])

cvpl_df= pd.DataFrame(cv_plc.todense(),columns=cv.get_feature_names(),index=rdf['Restaurant Name'])
cvpl_df.columns

Index(['bar', 'casualdining', 'dessertparlor', 'finedining', 'foodcourt',
       'kafe', 'kakilima', 'kios', 'lounge', 'quickbites', 'roti', 'tokokue',
       'tokominuman'],
      dtype='object')

In [28]:
cos_sin_plc=cosine_similarity(cv_plc)
pd.DataFrame(cos_sin_plc,index=rdf['Restaurant Name'],columns=rdf['Restaurant Name'])

0.0   
A Wen Seafood                                   1.000000          0.0   
Talaga Kuring                                   1.000000          0.0   
Tigalima Coffee & Kitchen                       0.707107          0.0   

Restaurant Name            Oma Recipe  Soto Betawi Bang Japri  Kopi Ranger  \
Restaurant Name                                                              
Holy Smokes                       0.0                     0.0          0.0   
Pempek Megaria                    0.0                     0.0          0.0   
Tjikinii Lima                     0.0                     0.0          0.0   
Gado-Gado Bon-Bin                 0.0                     0.0          0.0   
RM Wong Tegal                     0.0                     0.0          0.0   
...                               ...                     ...          ...   
Nock Nock Cafe & Resto            0.0                     0.0          0.0   
Abun Takoyaki                     0.0                     0.0          0.0   
A Wen Seafood                     0.0                     0.0          0.0   
Talaga Kuring                     0.0                     0.0          0.0   
Tigalima Coffee & Kitchen         0.0                     0.0          0.0   

Restaurant Name            Quick Chicken  Sate Tegal Kambing Muda 88  \
Restaurant Name                                                        
Holy Smokes                     0.000000                    0.000000   
Pempek Megaria                  1.000000                    1.000000   
Tjikinii Lima                   0.000000                    0.000000   
Gado-Gado Bon-Bin               1.000000                    1.000000   
RM Wong Tegal                   1.000000                    1.000000   
...                                  ...                         ...   
Nock Nock Cafe & Resto          0.707107                    0.707107   
Abun Takoyaki                   1.000000                    1.000000   
A Wen Seafood                   1.000000                    1.000000   
Talaga Kuring                   1.000000                    1.000000   
Tigalima Coffee & Kitchen       0.707107                    0.707107   

Restaurant Name            Rumah Makan Pak Ubung  FIFI Coffee Bar & Bistro  \
Restaurant Name                                                              
Holy Smokes                             0.000000                  0.000000   
Pempek Megaria                          1.000000                  0.000000   
Tjikinii Lima                           0.000000                  0.000000   
Gado-Gado Bon-Bin                       1.000000                  0.000000   
RM Wong Tegal                           1.000000                  0.000000   
...                                          ...                       ...   
Nock Nock Cafe & Resto                  0.707107                  0.707107   
Abun Takoyaki                           1.000000                  0.000000   
A Wen Seafood                           1.000000                  0.000000   
Talaga Kuring                           1.000000                  0.000000   
Tigalima Coffee & Kitchen               0.707107                  0.707107   

Restaurant Name            Nasi Goreng Karee Abang Gandul  Kedai Sutera  \
Restaurant Name                                                           
Holy Smokes                                      0.000000      0.000000   
Pempek Megaria                                   1.000000      1.000000   
Tjikinii Lima                                    0.000000      0.000000   
Gado-Gado Bon-Bin                                1.000000      1.000000   
RM Wong Tegal                                    1.000000      1.000000   
...                                                   ...           ...   
Nock Nock Cafe & Resto                           0.707107      0.707107   
Abun Takoyaki                                    1.000000      1.000000   
A Wen Seafood                                    1.00000

In [29]:
def get_recomendation_place(title):
    idx=rdf_csne[rdf_csne['Restaurant Name']==title].index[0]
    rest_similar=pd.Series(cos_sin_plc[idx])
    idx_similar=rest_similar.sort_values(ascending=False).head(6).index
    recommend=rdf_csne.loc[idx_similar]
    recommend.reset_index(drop=True,inplace=True)
    
    return recommend[1:6][['Restaurant Name', 'Place Type', 'Cuisines', 'Cuisine Regionality','Price for 2','Restaurant Address', 'Restaurant Rating','Restaurant Review','City', 'Province', 'Latitude', 'Longitude']]

In [30]:
get_recomendation_place('Cirengcrispi')

,Restaurant Name,Place Type,Cuisines,Cuisine Regionality,Price for 2,Restaurant Address,Restaurant Rating,Restaurant Review,City,Province,Latitude,Longitude
1,Coco Tree,"['Food Court', 'Kios']",['Minuman'],['Indonesia'],70000,"Mall of Indonesia, Lantai Lower Ground, Pasar ...",3.1,4,Jakarta Utara,DKI Jakarta,-6.150546,106.891921
2,Soto Ambengan,['Food Court'],['Soto'],"['Jawa', 'Indonesia']",150000,"Mall Kelapa Gading 3, Lantai 3, Eat & Eat, Jl....",3.3,6,Jakarta Utara,DKI Jakarta,-6.158510,106.907039
3,A&W Ice Cream,['Kios'],"['Ice Cream', 'Desserts']",['Indonesia'],50000,"ITC Permata Hijau, Lantai Ground, Jl. Letjen S...",3.4,4,Jakarta Selatan,DKI Jakarta,-6.220934,106.783784
4,Bim Coffee,['Food Court'],"['Kopi', 'Minuman']",['No Region'],100000,"Soekarno Hatta International Airport, Terminal...",2.8,4,Tangerang Kota,Banten,-6.131209,106.653184
5,Singapore Hainanese Chicken Rice,['Food Court'],['Ayam'],['Singapura'],100000,"Senayan City, Lantai 5, Delicaé, Jl. Asia Afri...",3.3,22,Jakarta Pusat,DKI Jakarta,-6.227689,106.797580


# 1.4 RECOMMENDER - PLACE (CITY)

In [31]:
rdf_csne['City STR'] = rdf_csne.apply(lambda row: minimize_str(row, 'City'), axis=1)

In [32]:
cv= CountVectorizer()
cv_city=cv.fit_transform(rdf_csne['City STR'])

In [33]:
cvct_df= pd.DataFrame(cv_city.todense(),columns=cv.get_feature_names(),index=rdf['Restaurant Name'])
cvct_df.columns

Index(['bekasikabupaten', 'bekasikota', 'bogorkabupaten', 'bogorkota',
       'cianjur', 'depok', 'jakartabarat', 'jakartapusat', 'jakartaselatan',
       'jakartatimur', 'jakartautara', 'tangerang', 'tangerangkabupaten',
       'tangerangkota', 'tangerangselatan'],
      dtype='object')

In [34]:
cos_sin_city=cosine_similarity(cv_city)
pd.DataFrame(cos_sin_city,index=rdf['Restaurant Name'],columns=rdf['Restaurant Name'])

& Resto                               0.0          0.0   
Abun Takoyaki                                        0.0          0.0   
A Wen Seafood                                        0.0          1.0   
Talaga Kuring                                        0.0          0.0   
Tigalima Coffee & Kitchen                            0.0          0.0   

Restaurant Name            Oma Recipe  Soto Betawi Bang Japri  Kopi Ranger  \
Restaurant Name                                                              
Holy Smokes                       1.0                     0.0          0.0   
Pempek Megaria                    0.0                     1.0          0.0   
Tjikinii Lima                     0.0                     1.0          0.0   
Gado-Gado Bon-Bin                 0.0                     1.0          0.0   
RM Wong Tegal                     0.0                     0.0          0.0   
...                               ...                     ...          ...   
Nock Nock Cafe & Resto            0.0                     0.0          0.0   
Abun Takoyaki                     1.0                     0.0          0.0   
A Wen Seafood                     0.0                     0.0          0.0   
Talaga Kuring                     0.0                     0.0          0.0   
Tigalima Coffee & Kitchen         0.0                     1.0          0.0   

Restaurant Name            Quick Chicken  Sate Tegal Kambing Muda 88  \
Restaurant Name                                                        
Holy Smokes                          0.0                         0.0   
Pempek Megaria                       0.0                         0.0   
Tjikinii Lima                        0.0                         0.0   
Gado-Gado Bon-Bin                    0.0                         0.0   
RM Wong Tegal                        0.0                         0.0   
...                                  ...                         ...   
Nock Nock Cafe & Resto               0.0                         0.0   
Abun Takoyaki                        0.0                         0.0   
A Wen Seafood                        0.0                         0.0   
Talaga Kuring                        0.0                         0.0   
Tigalima Coffee & Kitchen            0.0                         0.0   

Restaurant Name            Rumah Makan Pak Ubung  FIFI Coffee Bar & Bistro  \
Restaurant Name                                                              
Holy Smokes                                  0.0                       0.0   
Pempek Megaria                               0.0                       0.0   
Tjikinii Lima                                0.0                       0.0   
Gado-Gado Bon-Bin                            0.0                       0.0   
RM Wong Tegal                                0.0                       1.0   
...                                          ...                       ...   
Nock Nock Cafe & Resto                       0.0                       0.0   
Abun Takoyaki                                0.0                       0.0   
A Wen Seafood                                0.0                       1.0   
Talaga Kuring                                0.0                       0.0   
Tigalima Coffee & Kitchen                    0.0                       0.0   

Restaurant Name            Nasi Goreng Karee Abang Gandul  Kedai Sutera  \
Restaurant Name                                                           
Holy Smokes                                           0.0           0.0   
Pempek Megaria                                        0.0           0.0   
Tjikinii Lima                                         0.0           0.0   
Gado-Gado Bon-Bin                                     0.0           0.0   
RM Wong Tegal                                         0.0           0.0   
...                                                   ...           ...   
Nock Nock Cafe & Resto                                0.0           0.0   
Abun Ta

In [35]:
def get_recomendation_city(title):
    idx=rdf_csne[rdf_csne['Restaurant Name']==title].index[0]
    rest_similar=pd.Series(cos_sin_city[idx])
    idx_similar=rest_similar.sort_values(ascending=False).head(6).index
    recommend=rdf_csne.loc[idx_similar]
    recommend.reset_index(drop=True,inplace=True)
    
    return recommend[1:6][['Restaurant Name', 'Place Type', 'Cuisines', 'Cuisine Regionality','Price for 2','Restaurant Address', 'Restaurant Rating','Restaurant Review','City', 'Province', 'Latitude', 'Longitude']]

In [36]:
get_recomendation_city('Coco Tree')

,Restaurant Name,Place Type,Cuisines,Cuisine Regionality,Price for 2,Restaurant Address,Restaurant Rating,Restaurant Review,City,Province,Latitude,Longitude
1,Bakmie Asim,['Quick Bites'],['Bakmi'],['Indonesia'],70000,"Jl. Kelapa Kopyor Raya, Blok CE 2 No. 4, Kelap...",3.4,52,Jakarta Utara,DKI Jakarta,-6.169437,106.901691
2,Roastworks Coffee & Sisha,['Kafe'],"['Kopi', 'Minuman']","['No Region', 'Timur Tengah', 'Barat', 'Asia']",250000,"Rukan French Walk, Blok F No. 36, Jl. Kelapa G...",3.8,38,Jakarta Utara,DKI Jakarta,-6.150689,106.892830
3,Martabak Bandung Jaya,['Food Court'],"['Martabak', 'Makanan Jalanan']",['Indonesia'],100000,"Food Plaza, Jl Outer Ring Road, Pantai Indah K...",3.5,9,Jakarta Utara,DKI Jakarta,-6.104055,106.741430
4,Penang Hawker,['Casual Dining'],['Kopi'],"['Malaysia', 'Peranakan']",200000,"Rukan Emerald No. 1, Jl. Raya Pantai Indah Kap...",3.2,23,Jakarta Utara,DKI Jakarta,-6.105761,106.741031
5,Pawon Bamboe Solo,['Quick Bites'],['Kopi'],"['Jawa', 'Indonesia']",50000,"Jl. Boulevard Raya, Blok QJ 1, No. 21, Kelapa ...",3.3,5,Jakarta Utara,DKI Jakarta,-6.147323,106.909333


# 1.5 RECOMMENDER - PRICE

## PRICE SIMILARITY BASIC LOGIC & ALGORITHM

In [139]:
contoh  = [5000,6000,10000]
contohdf = pd.DataFrame(contoh, columns=['Price'])
contohdf

,Price
0,5000
1,6000
2,10000


In [140]:
prz_df = pd.DataFrame(index=contohdf.index, columns=contohdf.index)
prz_df

,0,1,2
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN


In [143]:
for index,row in prz_df.iterrows():
    for i in range(0,3):
        prz_df[index][i] = abs(contohdf['Price'][index] - contohdf['Price'][i])

prz_df

,0,1,2
0,0,1000,5000
1,1000,0,4000
2,5000,4000,0


## PUTTING IT TO REAL APPLICATION

In [148]:
prdf = rdf['Price for 2']/10000

In [154]:
prc_df = pd.DataFrame(index=rdf_csne.index, columns=rdf_csne.index)
prc_df

0    1    2    3    4    5    6    7    8    9    10   11   12   13    \
0     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
1     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
3     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
6779  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
6780  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
6781  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
6782  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
6783  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

     14   15   16   17   18   19   20   21   22   23   24   25   26   27    \
0     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
1     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
3     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
6779  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
6780  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
6781  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
6782  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
6783  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

     28   29   30   31   32   33   34   35   36   37   38   39   40   41    \
0     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
1     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
3     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
6779  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
6780  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
6781  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
6782  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
6783  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

     42   43   44   45   46   47   48   49   50   51   52   53   54   55    \
0     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
1     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
3     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
6779  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
6780  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
6781  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
6782  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
6783  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

     56   57   58   59   60   61   62   63   64   65   66   67   68   69    \
0     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
1     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2     NaN  NaN  NaN  Na

In [161]:
# for index,row in prc_df.iterrows():
#     for i in range(0,6784):
#         prc_df[index][i] = abs(prdf[index] - prdf[i])

In [162]:
prc_df = pd.read_csv("Price Matrix.csv")
prc_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,...,6284,6285,6286,6287,6288,6289,6290,6291,6292,6293,6294,6295,6296,6297,6298,6299,6300,6301,6302,6303,6304,6305,6306,6307,6308,6309,6310,6311,6312,6313,6314,6315,6316,6317,6318,6319,6320,6321,6322,6323,6324,6325,6326,6327,6328,6329,6330,6331,6332,6333,6334,6335,6336,6337,6338,6339,6340,6341,6342,6343,6344,6345,6346,6347,6348,6349,6350,6351,6352,6353,6354,6355,6356,6357,6358,6359,6360,6361,6362,6363,6364,6365,6366,6367,6368,6369,6370,6371,6372,6373,6374,6375,6376,6377,6378,6379,6380,6381,6382,6383,6384,6385,6386,6387,6388,6389,6390,6391,6392,6393,6394,6395,6396,6397,6398,6399,6400,6401,6402,6403,6404,6405,6406,6407,6408,6409,6410,6411,6412,6413,6414,6415,6416,6417,6418,6419,6420,6421,6422,6423,6424,6425,6426,6427,6428,6429,6430,6431,6432,6433,6434,6435,6436,6437,6438,6439,6440,6441,6442,6443,6444,6445,6446,6447,6448,6449,6450,6451,6452,6453,6454,6455,6456,6457,6458,6459,6460,6461,6462,6463,6464,6465,6466,6467,6468,6469,6470,6471,6472,6473,6474,6475,6476,6477,6478,6479,6480,6481,6482,6483,6484,6485,6486,6487,6488,6489,6490,6491,6492,6493,6494,6495,6496,6497,6498,6499,6500,6501,6502,6503,6504,6505,6506,6507,6508,6509,6510,6511,6512,6513,6514,6515,6516,6517,6518,6519,6520,6521,6522,6523,6524,6525,6526,6527,6528,6529,6530,6531,6532,6533,6534,6535,6536,6537,6538,6539,6540,6541,6542,6543,6544,6545,6546,6547,6548,6549,6550,6551,6552,6553,6554,6555,6556,6557,6558,6559,6560,6561,6562,6563,6564,6565,6566,6567,6568,6569,6570,6571,6572,6573,6574,6575,6576,6577,6578,6579,6580,6581,6582,6583,6584,6585,6586,6587,6588,6589,6590,6591,6592,6593,6594,6595,6596,6597,6598,6599,6600,6601,6602,6603,6604,6605,6606,6607,6608,6609,6610,6611,6612,6613,6614,6615,6616,6617,6618,6619,6620,6621,6622,6623,6624,6625,6626,6627,6628,6629,6630,6631,6632,6633,6634,6635,6636,6637,6638,6639,6640,6641,6642,6643,6644,6645,6646,6647,6648,6649,6650,6651,6652,6653,6654,6655,6656,6657,6658,6659,6660,6661,6662,6663,6664,6665,6666,6667,6668,6669,6670,6671,6672,6673,6674,6675,6676,6677,6678,6679,6680,6681,6682,6683,6684,6685,6686,6687,6688,6689,6690,6691,6692,6693,6694,6695,6696,6697,6698,6699,6700,6701,6702,6703,6704,

In [163]:
prc_df.to_csv("Price Matrix.csv")

In [214]:
prc_df2 = prc_df/(max(prc_df.max()))
prc_df2 = 1 - prc_df2
prc_df2

.919656  0.991392   
2     0.985653  0.942611  0.942611  0.928264  0.971306  0.985653  0.942611   
3     0.954089  0.997131  0.997131  0.988522  0.911047  0.925395  0.997131   
4     0.956958         1         1  0.985653  0.913917  0.928264         1   
...        ...       ...       ...       ...       ...       ...       ...   
6779  0.962697  0.994261  0.994261  0.979914  0.919656  0.934003  0.994261   
6780   0.94835  0.991392  0.991392  0.994261  0.905308  0.919656  0.991392   
6781  0.962697  0.994261  0.994261  0.979914  0.919656  0.934003  0.994261   
6782         1  0.956958  0.956958  0.942611  0.956958  0.971306  0.956958   
6783  0.962697  0.994261  0.994261  0.979914  0.919656  0.934003  0.994261   

          259       260       261       262       263       264       265   \
0     0.936872  0.942611  0.873745   0.89957         1  0.885222  0.913917   
1     0.939742  0.934003  0.997131  0.977044  0.876614  0.991392  0.962697   
2     0.994261         1  0.931133  0.956958  0.942611  0.942611  0.971306   
3     0.945481  0.939742  0.991392  0.982783  0.882353  0.997131  0.968436   
4      0.94835  0.942611  0.988522  0.985653  0.885222         1  0.971306   
...        ...       ...       ...       ...       ...       ...       ...   
6779  0.954089   0.94835  0.982783  0.991392  0.890961  0.994261  0.977044   
6780  0.939742  0.934003  0.997131  0.977044  0.876614  0.991392  0.962697   
6781  0.954089   0.94835  0.982783  0.991392  0.890961  0.994261  0.977044   
6782  0.991392  0.985653  0.945481  0.971306  0.928264  0.956958  0.985653   
6783  0.954089   0.94835  0.982783  0.991392  0.890961  0.994261  0.977044   

          266       267       268       269       270       271       272   \
0     0.902439         1  0.893831  0.928264  0.879484  0.956958  0.879484   
1     0.974175  0.876614  0.982783   0.94835  0.997131  0.919656  0.997131   
2     0.959828  0.942611   0.95122  0.985653  0.936872  0.985653  0.936872   
3     0.979914  0.882353  0.988522  0.954089  0.997131  0.925395  0.997131   
4     0.982783  0.885222  0.991392  0.956958  0.994261  0.928264  0.994261   
...        ...       ...       ...       ...       ...       ...       ...   
6779  0.988522  0.890961  0.997131  0.962697  0.988522  0.934003  0.988522   
6780  0.974175  0.876614  0.982783   0.94835  0.997131  0.919656  0.997131   
6781  0.988522  0.890961  0.997131  0.962697  0.988522  0.934003  0.988522   
6782  0.974175  0.928264  0.965567         1   0.95122  0.971306   0.95122   
6783  0.988522  0.890961  0.997131  0.962697  0.988522  0.934003  0.988522   

          273       274       275       276       277       278       279   \
0     0.882353  0.885222   0.89957  0.876614  0.913917  0.888092  0.890961   
1     0.994261  0.991392  0.977044         1  0.962697  0.988522  0.985653   
2     0.939742  0.942611  0.956958  0.934003  0.971306  0.945481   0.94835   
3            1  0.997131  0.982783  0.994261  0.968436  0.994261  0.991392   
4     0.997131         1  0.985653  0.991392  0.971306  0.997131  0.994261   
...        ...       ...       ...       ...       ...       ...       ...   
6779  0.991392  0.994261  0.991392  0.985653  0.977044  0.997131         1   
6780  0.994261  0.991392  0.977044         1  0.962697  0.988522  0.985653   
6781  0.991392  0.994261  0.991392  0.985653  0.977044  0.997131         1   
6782  0.954089  0.956958  0.971306   0.94835  0.985653  0.959828  0.962697   
6783  0.991392  0.994261  0.991392  0.985653  0.977044  0.997131         1   

          280       281       282       283       284       285       286   \
0     0.885222  0.956958  0.890961   0.28264  0.890961  0.942611  0.985653   
1     0.991392  0.919656  0.985653  0.159254  0.985653  0.934003  0.862267   
2     0.942611  0.985653   0.94835  0.225251   0.94835         1  0.928264   
3     0.997131  0.925395  0.991392  0.164993  0.991392  0.939742  0.868006   
4            1  0.928264  0.994261  0.167862  0.994261  0.942611  0.870875 

In [215]:
prc_arr = prc_df2.values

In [222]:
def get_recomendation_price(title):
    idx=rdf_csne[rdf_csne['Restaurant Name']==title].index[0]
    rest_similar=pd.Series(prc_df2[idx])
    idx_similar=rest_similar.sort_values(ascending=False).head(6).index
    recommend=rdf_csne.loc[idx_similar]
    recommend.reset_index(drop=True,inplace=True)
    
    return recommend[1:6][['Restaurant Name', 'Place Type', 'Cuisines', 'Cuisine Regionality','Price for 2','Restaurant Address', 'Restaurant Rating','Restaurant Review','City', 'Province', 'Latitude', 'Longitude']]

In [223]:
get_recomendation_price('Cirengcrispi')

,Restaurant Name,Place Type,Cuisines,Cuisine Regionality,Price for 2,Restaurant Address,Restaurant Rating,Restaurant Review,City,Province,Latitude,Longitude
1,Kedai Le Bye,['Quick Bites'],['Makanan Jalanan'],['Indonesia'],40000,"Jl. K.H.M. Usman No. 83, Beji, Depok",2.9,16,Depok,Jawa Barat,-6.364360,106.815648
2,Mie Ayam Bangka,['Quick Bites'],"['Ayam', 'Bakmi']","['Bangka', 'Indonesia']",40000,"Jl. Raya Hankam, Pondok Melati, Bekasi",3.1,8,Bekasi Kota,Jawa Barat,-6.273498,107.006597
3,Pastel Malinda,['Quick Bites'],['Kue & Roti'],['Indonesia'],40000,"Jl. Ratu Kemuning No. 26D, Green Ville, Jakarta",3.1,8,Jakarta Barat,DKI Jakarta,-6.169599,106.769330
4,Cakra Buana,['Food Court'],['Kopi'],"['Jawa', 'Indonesia']",40000,"Plaza Bapindo, Lantai Ground, MM Food, Jl. Jen...",3.1,5,Jakarta Selatan,DKI Jakarta,-6.225536,106.808113
5,Seafood Manado Om Islan,['Food Court'],['Seafood'],"['Manado', 'Asia', 'Indonesia']",40000,"Kawasan Kuliner BSM, Jl. MH Thamrin, Thamrin, ...",3.3,4,Jakarta Pusat,DKI Jakarta,-6.184639,106.824765


# 1.6 RECOMMENDER - ALL

In [234]:
def get_recomendation_all(title, rec_csne, rec_rgn, rec_plc, rec_city, rec_price):
    if rec_price == 0:
        idx=rdf_csne[rdf_csne['Restaurant Name']==title].index[0]
        cos_sin_all = ((cos_sin_csne*rec_csne) + (cos_sin_rgn*rec_rgn) + (cos_sin_plc*rec_plc) + (cos_sin_city*rec_city)) / (rec_csne + rec_rgn + rec_plc + rec_city)

    else:
        idx=rdf_csne[rdf_csne['Restaurant Name']==title].index[0]
        cos_sin_all = ((cos_sin_csne*rec_csne) + (cos_sin_rgn*rec_rgn) + (cos_sin_plc*rec_plc) + (cos_sin_city*rec_city) + (prc_arr*rec_price)) / (rec_csne + rec_rgn + rec_plc + rec_city + rec_price)

    rest_similar=pd.Series(cos_sin_all[idx])
    idx_similar=rest_similar.sort_values(ascending=False).head(6).index
    recommend=rdf_csne.loc[idx_similar]
    recommend.reset_index(drop=True,inplace=True)

    return recommend[1:6][['Restaurant Name', 'Place Type', 'Cuisines', 'Cuisine Regionality','Price for 2','Restaurant Address', 'Restaurant Rating','Restaurant Review','City', 'Province', 'Latitude', 'Longitude']]

In [238]:
get_recomendation_all('Sunday Folks Jakarta',1,1,1,1,1)

,Restaurant Name,Place Type,Cuisines,Cuisine Regionality,Price for 2,Restaurant Address,Restaurant Rating,Restaurant Review,City,Province,Latitude,Longitude
1,Afters,['Kafe'],"['Tea', 'Kopi', 'Snacks', 'Desserts']",['Barat'],150000,"Mall Kelapa Gading 3, Lantai 3, Jl. Kelapa Gad...",4.0,105,Jakarta Utara,DKI Jakarta,-6.156598,106.909951
2,Sam Gelato & Cafe,"['Kafe', 'Dessert Parlor']","['Desserts', 'Kopi', 'Ice Cream']",['No Region'],100000,"Taman Impian Jaya Ancol, Jl. Pintu Timur, Anco...",3.0,10,Jakarta Utara,DKI Jakarta,-6.122451,106.843700
3,Petals & Cream,"['Dessert Parlor', 'Kafe']","['Desserts', 'Kopi', 'Ice Cream']",['Indonesia'],70000,"Jl. Pluit Karang Utara Blok A4U No. 93, Pluit,...",3.7,28,Jakarta Utara,DKI Jakarta,-6.112211,106.779573
4,Krossroadz,"['Quick Bites', 'Kafe']","['Tea', 'Kopi', 'Snacks', 'Desserts']",['Indonesia'],200000,"Mahaka Square, Blok B8 No. 11, Jl. Kelapa Nias...",3.6,62,Jakarta Utara,DKI Jakarta,-6.149886,106.903349
5,Daun Kelapa,['Casual Dining'],['Kopi'],['Sunda'],200000,"Ruko Inkopal, Blok C No. 42-43A, Jl. Boulevard...",3.4,24,Jakarta Utara,DKI Jakarta,-6.151611,106.889248


# DEPLOY RECOMMENDER SYSTEM

In [227]:
pickle.dump(cv_csn, open('recom_cuisine.sav', 'wb'))
pickle.dump(cv_rgn, open('recom_cuisine_region.sav', 'wb'))
pickle.dump(cv_plc, open('recom_place.sav', 'wb'))
pickle.dump(cv_city, open('recom_city.sav', 'wb'))
np.save(open('recom_price.npy', 'wb'), prc_arr)

In [40]:
rdf_csne.drop(['Cuisines STR', 'Cuisine Regionality STR', 'Place Type STR', 'City STR'], axis=1).to_csv("RDF - Recommender System.csv")

In [6]:
prc_arr = np.load("recom_price.npy", allow_pickle=True)